In [ ]:
!pip install PySastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
!ls '../content/input'

abusive.csv  new_kamusalay.csv	re_dataset.csv	stopwordbahasa.csv


In [ ]:
data = pd.read_csv('../content/input/re_dataset.csv', encoding='latin-1')

alay_dict = pd.read_csv('../content/input/new_kamusalay.csv', encoding='latin-1', header=None)
alay_dict = alay_dict.rename(columns={0: 'original', 
                                      1: 'replacement'})

id_stopword_dict = pd.read_csv('../content/input/stopwordbahasa.csv', header=None)
id_stopword_dict = id_stopword_dict.rename(columns={0: 'stopword'})

In [ ]:
print("Shape: ", data.shape)
data.head(15)

In [ ]:
data.HS.value_counts()

0    7608
1    5561
Name: HS, dtype: int64

In [ ]:
print("Shape: ", alay_dict.shape)
alay_dict.head(15)

Shape:  (15167, 2)


,original,replacement
0,anakjakartaasikasik,anak jakarta asyik asyik
1,pakcikdahtua,pak cik sudah tua
2,pakcikmudalagi,pak cik muda lagi
3,t3tapjokowi,tetap jokowi
4,3x,tiga kali
5,aamiin,amin
6,aamiinn,amin
7,aamin,amin
8,aammiin,amin
9,abis,habis


In [ ]:
print("Shape: ", id_stopword_dict.shape)
id_stopword_dict.head()

Shape:  (758, 1)


,stopword
0,ada
1,adalah
2,adanya
3,adapun
4,agak


In [ ]:
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def lowercase(text):
    return text.lower()

def remove_unnecessary_char(text):
    text = re.sub('\n',' ',text) # Remove every '\n'
    text = re.sub('rt',' ',text) # Remove every retweet symbol
    text = re.sub('user',' ',text) # Remove every username
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))',' ',text) # Remove every URL
    text = re.sub('  +', ' ', text) # Remove extra spaces
    return text
    
def remove_nonaplhanumeric(text):
    text = re.sub('[^0-9a-zA-Z]+', ' ', text) 
    return text
    
def remove_num(text):
    no_digits = []

    for i in text:
      if not i.isdigit():
        no_digits.append(i)

    result = ''.join(no_digits)
    return result

alay_dict_map = dict(zip(alay_dict['original'], alay_dict['replacement']))
def normalize_alay(text):
    return ' '.join([alay_dict_map[word] if word in alay_dict_map else word for word in text.split(' ')])

def remove_stopword(text):
    text = ' '.join(['' if word in id_stopword_dict.stopword.values else word for word in text.split(' ')])
    text = re.sub('  +', ' ', text) # Remove extra spaces
    text = text.strip()
    return text

def stemming(text):
    return stemmer.stem(text)

print("remove_nonaplhanumeric: ", remove_nonaplhanumeric("Halooo,,,,, duniaa!!"))
print("remove_num: ", remove_num("Halo123"))
print("lowercase: ", lowercase("Halooo, duniaa!"))
print("stemming: ", stemming("Perekonomian Indonesia sedang dalam pertumbuhan yang membanggakan"))
print("remove_unnecessary_char: ", remove_unnecessary_char("Hehe\n\n RT USER USER apa kabs www.google.com\n  hehe"))
print("normalize_alay: ", normalize_alay("aamiin adek abis"))
print("remove_stopword: ", remove_stopword("ada hehe adalah huhu yang hehe"))

remove_nonaplhanumeric:  Halooo duniaa 
remove_num:  Halo
lowercase:  halooo, duniaa!
stemming:  ekonomi indonesia sedang dalam tumbuh yang bangga
remove_unnecessary_char:  Hehe RT USER USER apa kabs hehe
normalize_alay:  amin adik habis
remove_stopword:  hehe huhu hehe


In [ ]:
def preprocess(text):
    text = lowercase(text) # 1
    text = remove_nonaplhanumeric(text) # 2
    text = remove_num(text) # 2
    text = remove_unnecessary_char(text) # 2
    text = normalize_alay(text) # 3
    text = stemming(text) # 4
    text = remove_stopword(text) # 5
    return text

In [ ]:
data['Tweet'] = data['Tweet'].apply(preprocess)

In [ ]:
print("Shape: ", data.shape)
data.head(15)

Shape:  (13169, 13)


,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,cowok usaha lacak perhati gue lantas remeh per...,1,1,1,0,0,0,0,0,1,1,0,0
1,telat tau edan sarap gue gaul cigax jifla cal ...,0,1,0,0,0,0,0,0,0,0,0,0
2,kadang pikir percaya tuhan jatuh kali kali kad...,0,0,0,0,0,0,0,0,0,0,0,0
3,ku tau mata sipit lihat,0,0,0,0,0,0,0,0,0,0,0,0
4,kaum cebong kafir lihat dongok dungu haha,1,1,0,1,1,0,0,0,0,0,1,0
5,ya bani taplak kawan kawan xf xf x x xf xf x x...,1,1,0,1,0,0,0,0,1,0,1,0
6,deklarasi pilih kepala daerah aman anti hoaks ...,0,0,0,0,0,0,0,0,0,0,0,0
7,gue selesai re watch aldnoah zero kampret kara...,0,1,0,0,0,0,0,0,0,0,0,0
8,admin belanja po nak makan ais kepal milo ais ...,0,0,0,0,0,0,0,0,0,0,0,0
9,enak ngewe,0,1,0,0,0,0,0,0,0,0,0,0


In [ ]:
data.to_csv('preprocessed_indonesian.csv', index=False)

In [ ]:
FILENAME = "../content/preprocessed_indonesian.csv"          

col_list=['Tweet','HS']
missing=['nan']

dataset = pd.read_csv(FILENAME,usecols=col_list,encoding='latin-1',nrows=287)
dataset = dataset.dropna()
dataset


,Tweet,HS
0,cowok usaha lacak perhati gue lantas remeh per...,1
1,telat tau edan sarap gue gaul cigax jifla cal ...,0
2,kadang pikir percaya tuhan jatuh kali kali kad...,0
3,ku tau mata sipit lihat,0
4,kaum cebong kafir lihat dongok dungu haha,1
...,...,...
282,ya kali tentara nasional indonesia suruh urus ...,0
283,khawatir manuver politik bijak laku rezim moga...,0
284,madi arya lapor rocky gerung polisi serius kit...,0
285,bajing tengik,0


In [ ]:
sample=dataset['Tweet'].values.astype('U')

sample

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
idstopword="../content/stopwordbahasa.csv"
vectorizer = TfidfVectorizer()
response = vectorizer.fit_transform(sample)
print(response)

In [ ]:
vectorizer.get_feature_names_out()

In [ ]:
response.todense()

In [ ]:
df=pd.DataFrame(response.todense().T,
                index=vectorizer.get_feature_names_out(),
                columns=[f'D{i+1}' for i in range(len(sample))])
df

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(response, response)
print(cosine_sim)

In [ ]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import genesis
nltk.download('genesis')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
genesis_ic = wn.ic(genesis, False, 0.0)

import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
from sklearn.metrics import roc_auc_score

[nltk_data] Downloading package genesis to /root/nltk_data...
[nltk_data]   Package genesis is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
class KNN_NLC_Classifer():
    def __init__(self, k=15, distance_type = 'path'):
        self.k = k
        self.distance_type = distance_type

    # This function is used for training
    def fit(self, x_train, y_train):
        self.x_train = x_train
        self.y_train = y_train

    # This function runs the K(1) nearest neighbour algorithm and
    # returns the label with closest match. 
    def predict(self, x_test):
        self.x_test = x_test
        y_predict = []

        for i in range(len(x_test)):
            max_sim = 0
            max_index = 0
            for j in range(self.x_train.shape[0]):
                temp = self.document_similarity(x_test[i], self.x_train[j])
                if temp > max_sim:
                    max_sim = temp
                    max_index = j
            y_predict.append(self.y_train[max_index])
        return y_predict
    def convert_tag(self, tag):
        """Convert the tag given by nltk.pos_tag to the tag used by wordnet.synsets"""
        tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}
        try:
            return tag_dict[tag[0]]
        except KeyError:
            return None


    def doc_to_synsets(self, doc):
        tokens = word_tokenize(doc+' ')
        
        l = []
        tags = nltk.pos_tag([tokens[0] + ' ']) if len(tokens) == 1 else nltk.pos_tag(tokens)
        
        for token, tag in zip(tokens, tags):
            syntag = self.convert_tag(tag[1])
            syns = wn.synsets(token, syntag)
            if (len(syns) > 0):
                l.append(syns[0])
        return l  

    def similarity_score(self, s1, s2, distance_type = 'path'):
          """
          Calculate the normalized similarity score of s1 onto s2
          For each synset in s1, finds the synset in s2 with the largest similarity value.
          Sum of all of the largest similarity values and normalize this value by dividing it by the
          number of largest similarity values found.

          Args:
              s1, s2: list of synsets from doc_to_synsets

          Returns:
              normalized similarity score of s1 onto s2
          """
          s1_largest_scores = []

          for i, s1_synset in enumerate(s1, 0):
              max_score = 0
              for s2_synset in s2:
                  if distance_type == 'path':
                      score = s1_synset.path_similarity(s2_synset, simulate_root = False)
                  else:
                      score = s1_synset.wup_similarity(s2_synset)                  
                  if score != None:
                      if score > max_score:
                        max_score = score
              
              if max_score != 0:
                  s1_largest_scores.append(max_score)
          
          mean_score = np.mean(s1_largest_scores)
                 
          return mean_score  
    def document_similarity(self,doc1, doc2):
          """Finds the symmetrical similarity between doc1 and doc2"""

          synsets1 = self.doc_to_synsets(doc1)
          synsets2 = self.doc_to_synsets(doc2)
          
          return (self.similarity_score(synsets1, synsets2) + self.similarity_score(synsets2, synsets1)) / 2


In [ ]:
doc1 = 'I like rains'
doc2 = 'I like showers'
x = KNN_NLC_Classifer()
print("Test Similarity Score: ", x.document_similarity(doc1, doc2))

Test Similarity Score:  0.6946386946386947


In [ ]:
X_train=dataset['Tweet']
y_train=dataset["HS"]
print(X_train)

0      cowok usaha lacak perhati gue lantas remeh per...
1      telat tau edan sarap gue gaul cigax jifla cal ...
2      kadang pikir percaya tuhan jatuh kali kali kad...
3                                ku tau mata sipit lihat
4              kaum cebong kafir lihat dongok dungu haha
                             ...                        
282    ya kali tentara nasional indonesia suruh urus ...
283    khawatir manuver politik bijak laku rezim moga...
284    madi arya lapor rocky gerung polisi serius kit...
285                                        bajing tengik
286     pakai buruh cina rakyat anggur dasar rejim kodok
Name: Tweet, Length: 287, dtype: object


In [ ]:
# 4. Train the Classifier
classifier = KNN_NLC_Classifer(k=15, distance_type='path')
classifier.fit(X_train, y_train)

final_test_list = ['Dasar anjing gak guna jokowi','Bangsat kamu kek monyet','Selamat pagi semuanya','Halo selamat pagi','Bangun pagi nihh',
                   'Cina goblok gabisa kerja','Bumi berputar dalam matahari']
test_corpus = []
for i in range(len(final_test_list)):
    temp= preprocess(final_test_list[i])
    test_corpus.append(temp)
test_corpus

['dasar anjing jokowi',
 'bangsat kayak monyet',
 'selamat pagi',
 'halo selamat pagi',
 'bangun pagi nih',
 'cina goblok kerja',
 'bumi putar matahari']

In [ ]:
y_pred_final = classifier.predict(test_corpus)

output_df = pd.DataFrame(data = {'Tweet': final_test_list, 'code': y_pred_final})
output_df['answer'] = np.where(output_df['code']==1, 'HS','NonHS')
print(output_df)

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


                          Tweet  code answer
0  Dasar anjing gak guna jokowi     1     HS
1       Bangsat kamu kek monyet     0  NonHS
2         Selamat pagi semuanya     1     HS
3             Halo selamat pagi     0  NonHS
4              Bangun pagi nihh     0  NonHS
5      Cina goblok gabisa kerja     1     HS
6  Bumi berputar dalam matahari     1     HS
